In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# import os

# # Path to your zip file
# zip_path = '/content/drive/MyDrive/actionRecognition/crime16.zip'

# # Create extraction directory (if it doesn't exist)
# extract_path = '/content/drive/MyDrive/actionRecognition/crime16'
# os.makedirs(extract_path, exist_ok=True)

# # Unzip the file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print(f"Files extracted to: {extract_path}")

In [ ]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys

<ipython-input-2-a0647214ad17>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
!git clone https://github.com/sanchit2843/Videoclassification
%cd Videoclassification

Cloning into 'Videoclassification'...
remote: Enumerating objects: 51, done.
remote: Total 51 (delta 0), reused 0 (delta 0), pack-reused 51 (from 1)
Receiving objects: 100% (51/51), 100.12 KiB | 5.27 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/Videoclassification


In [ ]:
#Label file:
data_path = '/content/drive/MyDrive/actionRecognition/crime16'
classes = os.listdir(data_path)
print(classes)
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

['Robbery', 'Vandalism', 'Shoplifting', 'normal', 'Burglary', 'Stealing']


In [ ]:
id = list()
path = '/content/drive/MyDrive/actionRecognition/crime16'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    id.append((i,p2))

In [ ]:
class video_dataset(Dataset):
    def __init__(self,frame_list,sequence_length = 16,transform = None):
        self.frame_list = frame_list
        self.transform = transform
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.frame_list)
    def __getitem__(self,idx):
        label,path = self.frame_list[idx]
        img = cv2.imread(path)
        seq_img = list()
        for i in range(16):
          img1 = img[:,128*i:128*(i+1),:]
          if(self.transform):
            img1 = self.transform(img1)
          seq_img.append(img1)
        seq_image = torch.stack(seq_img)
        seq_image = seq_image.reshape(3,16,im_size,im_size)
        return seq_image,decoder[label]

In [ ]:
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torch

# Parameters
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]
validation_split = 0.2
batch_size = 8

# Define transforms
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# For validation, we don't need augmentation transforms
val_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create full dataset
full_dataset = video_dataset(id, sequence_length=16, transform=train_transforms)

# Calculate lengths for split
total_size = len(full_dataset)
val_size = int(validation_split * total_size)
train_size = total_size - val_size

# Split the dataset
train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)  # for reproducibility
)

# Override transforms for validation dataset
val_dataset.dataset.transform = val_transforms

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=4,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    num_workers=4,
    shuffle=False  # No need to shuffle validation data
)

# Create dataloaders dictionary
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

print(f"Training set size: {train_size}")
print(f"Validation set size: {val_size}")

Training set size: 4000
Validation set size: 1000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from model import resnet50
model = resnet50(class_num=6).to('cpu')

ModuleNotFoundError: No module named 'model'

In [ ]:
from clr import *
device = 'cpu'
cls_criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum = 0.9,weight_decay = 1e-4)
num_epochs = 20
onecyc = OneCycle(len(train_loader)*num_epochs,1e-3)

In [ ]:
# os.makedirs('/content/drive/MyDrive/actionRecognition/weights_crime',exist_ok = True)
# from torch.autograd import Variable
# iteration = 0
# acc_all = list()
# loss_all = list()

# for epoch in range(num_epochs):
#     print('')
#     print(f"--- Epoch {epoch} ---")
#     phase1 = dataloaders.keys()
#     for phase in phase1:
#         print('')
#         print(f"--- Phase {phase} ---")
#         epoch_metrics = {"loss": [], "acc": []}
#         for batch_i, (X, y) in enumerate(dataloaders[phase]):
#             #iteration = iteration+1
#             image_sequences = Variable(X.to(device), requires_grad=True)
#             labels = Variable(y.to(device), requires_grad=False)
#             optimizer.zero_grad()
#             #model.lstm.reset_hidden_state()
#             predictions = model(image_sequences)
#             loss = cls_criterion(predictions, labels)
#             acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
#             loss.backward()
#             optimizer.step()
#             epoch_metrics["loss"].append(loss.item())
#             epoch_metrics["acc"].append(acc)
#             if(phase=='train'):
#                 lr,mom = onecyc.calc()
#                 update_lr(optimizer, lr)
#                 update_mom(optimizer, mom)
#             batches_done = epoch * len(dataloaders[phase]) + batch_i
#             batches_left = num_epochs * len(dataloaders[phase]) - batches_done
#             sys.stdout.write(
#                     "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
#                     % (
#                         epoch,
#                         num_epochs,
#                         batch_i,
#                         len(dataloaders[phase]),
#                         loss.item(),
#                         np.mean(epoch_metrics["loss"]),
#                         acc,
#                         np.mean(epoch_metrics["acc"]),
#                     )
#                 )

#                 # Empty cache
#             if torch.cuda.is_available():
#                 torch.cuda.empty_cache()

#         print('')
#         print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
#         torch.save(model.state_dict(),'/content/drive/MyDrive/actionRecognition/weights_crime/c3d_{}.h5'.format(epoch))
#         if(phase=='train'):
#           acc_all.append(np.mean(epoch_metrics["acc"]))
#           loss_all.append(np.mean(epoch_metrics["loss"]))

In [ ]:
def error_plot(loss):
    plt.figure(figsize=(10,5))
    plt.plot(loss)
    plt.title("Training loss plot")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.show()
def acc_plot(acc):
    plt.figure(figsize=(10,5))
    plt.plot(acc)
    plt.title("Training accuracy plot")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.show()

In [ ]:
import os
import sys
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.amp import autocast, GradScaler

def train_and_evaluate(model, dataloaders, optimizer, num_epochs, device, onecyc):
    os.makedirs('/content/drive/MyDrive/actionRecognition/weights_crime', exist_ok=True)

    # Initialize scaler for mixed precision training
    scaler = GradScaler()

    # Track metrics
    acc_all = []
    loss_all = []
    best_acc = 0.0
    best_model_path = ''

    # Loss criterion
    cls_criterion = nn.CrossEntropyLoss()

    model.to(device)

    for epoch in range(num_epochs):
        print(f"\n--- Epoch {epoch}/{num_epochs-1} ---")

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_metrics = {"loss": [], "acc": []}

            # Enable gradient computation only during training
            with torch.set_grad_enabled(phase == 'train'):
                for batch_i, (X, y) in enumerate(dataloaders[phase]):
                    # Move data to GPU
                    image_sequences = X.to(device)
                    labels = y.to(device)

                    # Clear gradients
                    optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()

                    # Mixed precision training
                    with autocast():
                        predictions = model(image_sequences)
                        loss = cls_criterion(predictions, labels)

                    # Calculate accuracy
                    acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()

                    if phase == 'train':
                        # Scale loss and backward pass
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                        # Update learning rate and momentum
                        lr, mom = onecyc.calc()
                        update_lr(optimizer, lr)
                        update_mom(optimizer, mom)

                    # Track metrics
                    epoch_metrics["loss"].append(loss.item())
                    epoch_metrics["acc"].append(acc)

                    # Progress update
                    batches_done = epoch * len(dataloaders[phase]) + batch_i
                    batches_left = num_epochs * len(dataloaders[phase]) - batches_done

                    sys.stdout.write(
                        "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                        % (
                            epoch,
                            num_epochs,
                            batch_i,
                            len(dataloaders[phase]),
                            loss.item(),
                            np.mean(epoch_metrics["loss"]),
                            acc,
                            np.mean(epoch_metrics["acc"]),
                        )
                    )

            # Epoch summary
            epoch_acc = np.mean(epoch_metrics["acc"])
            epoch_loss = np.mean(epoch_metrics["loss"])
            print(f'\n{phase.capitalize()} Epoch {epoch} - Acc: {epoch_acc:.2f}%, Loss: {epoch_loss:.4f}')

            # Save metrics for training phase
            if phase == 'train':
                acc_all.append(epoch_acc)
                loss_all.append(epoch_loss)

            # Save best model on validation
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_path = f'/content/drive/MyDrive/actionRecognition/weights_crime/c3d_best.h5'
                torch.save(model.state_dict(), best_model_path)

            # Save regular checkpoint
            if phase == 'train':
                torch.save(model.state_dict(),
                         f'/content/drive/MyDrive/actionRecognition/weights_crime/c3d_{epoch}.h5')

        # Clear GPU cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Final evaluation
    print("\nPerforming final evaluation...")
    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    eval_metrics = {"loss": [], "acc": []}

    with torch.no_grad():
        for X, y in dataloaders['val']:
            image_sequences = X.to(device)
            labels = y.to(device)

            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.argmax(1) == labels).cpu().numpy().mean()

            eval_metrics["loss"].append(loss.item())
            eval_metrics["acc"].append(acc)

    final_acc = np.mean(eval_metrics["acc"])
    final_loss = np.mean(eval_metrics["loss"])

    print(f"\nFinal Evaluation Results:")
    print(f"Best Validation Accuracy: {best_acc:.2f}%")
    print(f"Final Test Accuracy: {final_acc:.2f}%")
    print(f"Final Test Loss: {final_loss:.4f}")

    return {
        'training_acc': acc_all,
        'training_loss': loss_all,
        'best_acc': best_acc,
        'final_acc': final_acc,
        'best_model_path': best_model_path
    }

In [ ]:
# Example usage
# torch.amp.GradScaler('cuda',
results = train_and_evaluate(
    model=model,
    dataloaders=dataloaders,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    onecyc=onecyc
)

# Access results
print(f"Best validation accuracy: {results['best_acc']:.2f}%")
print(f"Best model saved at: {results['best_model_path']}")

In [ ]:
loss_all

In [ ]:
error_plot(loss_all)

In [ ]:
acc_plot(acc_all)

In [ ]:
import torch
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
from torch.amp import autocast, GradScaler

def load_and_preprocess_video(video_path, sequence_length=16, im_size=128):
    """Load and preprocess video for inference"""
    mean = [0.4889, 0.4887, 0.4891]
    std = [0.2074, 0.2074, 0.2074]

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    # Read video
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
        frame_count += 1

    cap.release()

    # Handle videos shorter than sequence_length
    if frame_count < sequence_length:
        last_frame = frames[-1]
        for _ in range(sequence_length - frame_count):
            frames.append(last_frame)

    # Sample sequence_length frames uniformly
    indices = np.linspace(0, len(frames) - 1, sequence_length, dtype=int)
    frames = [frames[i] for i in indices]

    # Apply transforms
    frames = [transform(Image.fromarray(frame)) for frame in frames]

    # Stack frames
    frames = torch.stack(frames)

    return frames.unsqueeze(0)  # Add batch dimension

def classify_video(model_path, video_path, device='cuda', class_labels=None):
    """
    Load model and classify a video

    Args:
        model_path: Path to the .h5 model weights
        video_path: Path to the video file
        device: 'cuda' or 'cpu'
        class_labels: List of class names (optional)
    """
    # Load the model architecture (you need to define this based on your model)
    # model = YourModelClass()  # Replace with your model architecture
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()

    # Preprocess video
    input_tensor = load_and_preprocess_video(video_path)
    input_tensor = input_tensor.to(device)

    # Inference
    with torch.no_grad():
        predictions = model(input_tensor)
        probabilities = torch.nn.functional.softmax(predictions, dim=1)

        # Get the predicted class
        pred_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][pred_class].item()

    # Format results
    result = {
        'predicted_class_idx': pred_class,
        'confidence': confidence * 100,  # Convert to percentage
        'class_name': class_labels[pred_class] if class_labels else f"Class {pred_class}",
        'probabilities': probabilities[0].cpu().numpy()
    }

    return result

# Example usage:
if __name__ == "__main__":
    # Define your class labels
    class_labels = ['Robbery', 'Vandalism', 'Shoplifting', 'normal', 'Burglary', 'Stealing']

    # Configuration
    model_path = '/content/drive/MyDrive/actionRecognition/weights_crime/c3d_best.h5'
    video_path = '/content/drive/MyDrive/actionRecognition/anomaly_videos/Stealing/Stealing002_x264.mp4'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Classify video
    result = classify_video(
        model_path=model_path,
        video_path=video_path,
        device=device,
        class_labels=class_labels
    )

    # Print results
    print(f"\nPrediction Results:")
    print(f"Predicted Class: {result['class_name']}")
    print(f"Confidence: {result['confidence']:.2f}%")

    # Print all class probabilities
    if class_labels:
        print("\nClass Probabilities:")
        for i, prob in enumerate(result['probabilities']):
            print(f"{class_labels[i]}: {prob*100:.2f}%")

UnboundLocalError: cannot access local variable 'model' where it is not associated with a value

In [ ]:
from model import resnet50
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
from clr import OneCycle

def load_and_preprocess_video(video_path, sequence_length=16, im_size=128):
    """Load and preprocess video for inference"""
    mean = [0.4889, 0.4887, 0.4891]
    std = [0.2074, 0.2074, 0.2074]

    transform = transforms.Compose([
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    # Read video
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frames.append(frame)
        frame_count += 1

    cap.release()

    # Handle videos shorter than sequence_length
    if frame_count < sequence_length:
        last_frame = frames[-1]
        for _ in range(sequence_length - frame_count):
            frames.append(last_frame)

    # Sample sequence_length frames uniformly
    indices = np.linspace(0, len(frames) - 1, sequence_length, dtype=int)
    frames = [frames[i] for i in indices]

    # Apply transforms
    transformed_frames = [transform(frame) for frame in frames]

    # Stack frames along a new dimension (temporal)
    # Change from [T, C, H, W] to [C, T, H, W]
    frames_tensor = torch.stack(transformed_frames, dim=1)  # Changed from dim=0 to dim=1

    return frames_tensor.unsqueeze(0)  # Add batch dimension [B, C, T, H, W]


def classify_video(model_path, video_path, device='cuda', class_labels=None):
    """Load ResNet50 model and classify a video"""
    # Initialize model
    model = resnet50(class_num=len(class_labels)).to(device)  # Update to match number of classes
    # model = resnet50(class_num=6).to(device)  # Update to match number of classes

    # Load weights
    # model.load_state_dict(torch.load(model_path))
    #  map_location='cpu'

    # model.load_state_dict(torch.load(model_path, weights_only=True))
    model.load_state_dict(torch.load(model_path,     map_location=device, weights_only=True))
    model.eval()

    # Preprocess video
    input_tensor = load_and_preprocess_video(video_path)
    input_tensor = input_tensor.to(device)

    # Inference
    with torch.no_grad():
        predictions = model(input_tensor)
        probabilities = torch.nn.functional.softmax(predictions, dim=1)

        pred_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][pred_class].item()

    result = {
        'predicted_class_idx': pred_class,
        'confidence': confidence * 100,
        'class_name': class_labels[pred_class] if class_labels else f"Class {pred_class}",
        'probabilities': probabilities[0].cpu().numpy()
    }

    return result

# Example usage
if __name__ == "__main__":
    # Your actual classes
    class_labels = ['Robbery', 'Vandalism', 'Shoplifting', 'normal', 'Burglary', 'Stealing']

    # Configuration
    model_path = '/content/drive/MyDrive/actionRecognition/weights_crime/c3d_best_v1.h5'
    # video_path = '/content/drive/MyDrive/actionRecognition/anomaly_videos/Stealing/Stealing016_x264.mp4'
    video_path = '/content/4fcf293d-7d6a-4da0-bcef-c9b5dd6404f6.mp4'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Classify video
    try:
        result = classify_video(
            model_path=model_path,
            video_path=video_path,
            device=device,
            class_labels=class_labels
        )

        # Print results
        print(f"\nPrediction Results:")
        print(f"Predicted Class: {result['class_name']}")
        print(f"Confidence: {result['confidence']:.2f}%")

        # Print all class probabilities
        print("\nClass Probabilities:")
        for i, prob in enumerate(result['probabilities']):
            if i < len(class_labels):  # Make sure we don't exceed the number of labels
                print(f"{class_labels[i]}: {prob*100:.2f}%")
    except Exception as e:
        print(f"Error occurred: {str(e)}")


Prediction Results:
Predicted Class: Stealing
Confidence: 100.00%

Class Probabilities:
Robbery: 0.00%
Vandalism: 0.00%
Shoplifting: 0.00%
normal: 0.00%
Burglary: 0.00%
Stealing: 100.00%


In [ ]:
!pip install opencv-python pillow

In [ ]:
!pwd

/content/Videoclassification


In [ ]:
from model import resnet50
import torch
import torch.nn as nn
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
import time
from collections import deque
import logging

class RealTimeActionDetector:
    def __init__(self, model_path, class_labels, device='cuda',
                 sequence_length=16, im_size=128,
                 confidence_threshold=50.0):
        """
        Initialize the real-time action detection system

        :param model_path: Path to the trained model weights
        :param class_labels: List of class labels
        :param device: Computing device (cuda/cpu)
        :param sequence_length: Number of frames to process
        :param im_size: Image resize dimension
        :param confidence_threshold: Minimum confidence to trigger an alert
        """
        # Model and preprocessing setup
        self.device = device if torch.cuda.is_available() else 'cpu'
        self.class_labels = class_labels
        self.sequence_length = sequence_length
        self.im_size = im_size
        self.confidence_threshold = confidence_threshold

        # Initialize model
        self.model = self._initialize_model(model_path)

        # Transform setup
        self.transform = self._create_transform()

        # Logging setup
        logging.basicConfig(level=logging.INFO,
                            format='%(asctime)s - %(levelname)s: %(message)s')
        self.logger = logging.getLogger(__name__)

        # Frame buffer
        self.frame_buffer = deque(maxlen=sequence_length)

    def _initialize_model(self, model_path):
        """Initialize the model"""
        model = resnet50(class_num=len(self.class_labels)).to(self.device)
        model.load_state_dict(torch.load(model_path,
                                         map_location=self.device,
                                         weights_only=True))
        model.eval()
        return model

    def _create_transform(self):
        """Create image transformation pipeline"""
        mean = [0.4889, 0.4887, 0.4891]
        std = [0.2074, 0.2074, 0.2074]
        return transforms.Compose([
            transforms.Resize((self.im_size, self.im_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def _process_frame_sequence(self, frames):
        """Process a sequence of frames"""
        transformed_frames = [
            self.transform(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            for frame in frames
        ]
        frames_tensor = torch.stack(transformed_frames, dim=1)
        return frames_tensor.unsqueeze(0).to(self.device)

    def detect_action(self, frame_buffer):
        """
        Detect action in the frame sequence

        :param frame_buffer: List of frames
        :return: Detection results
        """
        with torch.no_grad():
            input_tensor = self._process_frame_sequence(list(frame_buffer))
            predictions = self.model(input_tensor)
            probabilities = torch.nn.functional.softmax(predictions, dim=1)

            # Get top 2 predictions
            top2_probs, top2_indices = torch.topk(probabilities[0], k=2)

            results = {
                'top_class': {
                    'name': self.class_labels[top2_indices[0]],
                    'probability': top2_indices[0].item() * 100
                },
                'second_class': {
                    'name': self.class_labels[top2_indices[1]],
                    'probability': top2_indices[1].item() * 100
                }
            }

            return results

    def process_camera_feed(self, camera_id=0):
        """
        Real-time camera feed processing

        :param camera_id: Camera index or stream URL
        """
        cap = cv2.VideoCapture(camera_id)

        if not cap.isOpened():
            self.logger.error("Could not open camera")
            return

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Add frame to buffer
                self.frame_buffer.append(frame.copy())

                # Process when buffer is full
                if len(self.frame_buffer) == self.sequence_length:
                    detection_results = self.detect_action(self.frame_buffer)

                    # Check if detection meets confidence threshold
                    if (detection_results['top_class']['probability'] >= self.confidence_threshold and
                        detection_results['top_class']['name'] != 'normal'):

                        # Log and potentially trigger alert
                        alert_message = (
                            f"ALERT: Detected {detection_results['top_class']['name']} "
                            f"(Prob: {detection_results['top_class']['probability']:.2f}%) "
                            f"with {detection_results['second_class']['name']} "
                            f"(Prob: {detection_results['second_class']['probability']:.2f}%)"
                        )

                        self.logger.warning(alert_message)

                        # Here you can add email notification function call
                        # send_email(alert_message)

                # Display frame with detection info
                cv2.putText(frame,
                            f"Top: {detection_results['top_class']['name']} "
                            f"({detection_results['top_class']['probability']:.2f}%)",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                cv2.putText(frame,
                            f"Second: {detection_results['second_class']['name']} "
                            f"({detection_results['second_class']['probability']:.2f}%)",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                cv2.imshow('Real-time Action Detection', frame)

                # Break on 'q' press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            cap.release()
            cv2.destroyAllWindows()

def main():
    # Configuration
    class_labels = ['Robbery', 'Vandalism', 'Shoplifting', 'normal', 'Burglary', 'Stealing']
    model_path = '/content/drive/MyDrive/actionRecognition/weights_crime/c3d_best_v1.h5'

    # Initialize detector
    detector = RealTimeActionDetector(
        model_path=model_path,
        class_labels=class_labels,
        confidence_threshold=50.0  # Adjust as needed
    )

    # Start real-time processing
    detector.process_camera_feed()

if __name__ == "__main__":
    main()


ERROR:__main__:Could not open camera
